In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@databricksadlsete.dfs.core.windows.net/orders")

In [0]:
df.printSchema()

In [0]:
df.display()             #_rescued_data it’s a special column automatically added by Auto Loader when it encounters extra fields in the source data that aren’t part of the defined schema


In [0]:
df = df.withColumnRenamed("_rescued_data","rescued_data")

In [0]:
df=df.drop("rescued_data")
df.display()

### **date transformations**

In [0]:
df = df.withColumn(
    "order_date",
    to_timestamp(col("order_date"))
)

df.display()

In [0]:
df = df.withColumn("Year", year(col("order_date")))

df.display()

In [0]:
#df = df.withColumn("Month", month(col("order_date")))

#df.display()

### **Windows Functions**

In [0]:
df1 = df.withColumn("flag",dense_rank().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
display(df1)
                    

In [0]:
df1 = df1.withColumn("rank_flag",rank().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
display(df1)

In [0]:
df1 = df1.withColumn("row_flag",row_number().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
display(df1)

### **Classes-OOPS**

In [0]:
class windows:

  def dense_rank(self,df):
    df_dense_rank=df.withColumn("flag",dense_rank().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
    return df_dense_rank

  def rank(self,df):
    df_rank=df.withColumn("rank_flag",rank().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))  
    return df_rank

  def row_number(self,df):
    df_row_number=df.withColumn("row_flag",row_number().over(Window.partitionBy("Year").orderBy(desc("total_amount"))))
    return df_row_number


In [0]:
df_new=df

In [0]:
df_new.display()

In [0]:
obj = windows()

In [0]:
df_result=obj.dense_rank(df_new)

df_result.display()

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksadlsete.dfs.core.windows.net/order")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_ete_cata.silver.Orders_Silver
USING DELTA
LOCATION 'abfss://silver@databricksadlsete.dfs.core.windows.net/orders'

In [0]:
#%sql
#select * from databricks_ete_cata.silver.Orders_Silver